## Import package

In [1]:
import pandas as pd
import os
import numpy as np
from collections import defaultdict
from tqdm import tqdm

pd.options.mode.chained_assignment = None

In [2]:
prefix_path = '../data/mimiciii/1.4/'

## Split CHARTEVENTS to several dataset

In [7]:
count = 1
for reader in tqdm(pd.read_csv(prefix_path + 'CHARTEVENTS.csv', chunksize = 40000000)):
    reader.to_csv(prefix_path + 'CHARTEVENTS' + str(count) + '.csv')
    count += 1

0it [00:00, ?it/s]/home/toooot/anaconda3/envs/sepsis/lib/python3.8/site-packages/tqdm/std.py:1195: DtypeWarning: Columns (8,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
4it [07:27, 110.95s/it]/home/toooot/anaconda3/envs/sepsis/lib/python3.8/site-packages/tqdm/std.py:1195: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
6it [11:11, 111.30s/it]/home/toooot/anaconda3/envs/sepsis/lib/python3.8/site-packages/tqdm/std.py:1195: DtypeWarning: Columns (8,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
9it [15:09, 101.08s/it]


## Load Data

In [3]:
paths = sorted(os.listdir(prefix_path))
remove_paths = []
for path in paths:
    if path[-4:] != '.csv':
        remove_paths.append(path)
for path in remove_paths:
    paths.remove(path)
paths.remove('CALLOUT.csv')
paths.remove('CAREGIVERS.csv')
paths.remove('CPTEVENTS.csv')
paths.remove('DATETIMEEVENTS.csv')
paths.remove('SERVICES.csv',)
paths.remove('TRANSFERS.csv')
paths.remove('PROCEDURES_ICD.csv')
paths.remove('PROCEDUREEVENTS_MV.csv')
paths.remove('MICROBIOLOGYEVENTS.csv')
paths.remove('D_CPT.csv')
paths

['ADMISSIONS.csv',
 'CHARTEVENTS1.csv',
 'CHARTEVENTS2.csv',
 'CHARTEVENTS3.csv',
 'CHARTEVENTS4.csv',
 'CHARTEVENTS5.csv',
 'CHARTEVENTS6.csv',
 'CHARTEVENTS7.csv',
 'CHARTEVENTS8.csv',
 'CHARTEVENTS9.csv',
 'DIAGNOSES_ICD.csv',
 'DIAGNOSES_ICD_FULL.csv',
 'DRGCODES.csv',
 'D_ICD_DIAGNOSES.csv',
 'D_ICD_DIAGNOSES_FULL.csv',
 'D_ICD_PROCEDURES.csv',
 'D_ITEMS.csv',
 'D_LABITEMS.csv',
 'ICUSTAYS.csv',
 'INPUTEVENTS_CV.csv',
 'INPUTEVENTS_MV.csv',
 'LABEVENTS.csv',
 'NOTEEVENTS.csv',
 'OUTPUTEVENTS.csv',
 'PATIENTS.csv',
 'PRESCRIPTIONS.csv']

In [4]:
datas = []
for path in tqdm(paths):
    datas.append(pd.read_csv(prefix_path + path))

  0%|                                                                                                                                                                               | 0/26 [00:00<?, ?it/s]/tmp/ipykernel_3321/136570088.py:3: DtypeWarning: Columns (10,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  datas.append(pd.read_csv(prefix_path + path))
 19%|████████████████████████████████                                                                                                                                       | 5/26 [00:21<01:33,  4.43s/it]/tmp/ipykernel_3321/136570088.py:3: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  datas.append(pd.read_csv(prefix_path + path))
 27%|████████████████████████████████████████████▉                                                                                                                          | 7/26 [00:32<01:32,  4.86s/it]/tmp/ipykernel_

## Clean Data to remain SEPSIS patient and sort Data

In [5]:
i = paths.index('D_ICD_DIAGNOSES.csv')
D_ICD_DIAGNOSES = datas[i].copy()
'''
D_ICD_DIAGNOSES.drop(D_ICD_DIAGNOSES[['sepsis' not in s.lower() for s in D_ICD_DIAGNOSES['LONG_TITLE']]].index, inplace=True)
D_ICD_DIAGNOSES.to_csv(prefix_path + paths[i])
'''
D_ICD_DIAGNOSES

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,0,9049,9049,9050,77181,NB septicemia [sepsis],Septicemia [sepsis] of newborn
1,1,10304,10304,11403,99591,Sepsis,Sepsis
2,2,10305,10305,11404,99592,Severe sepsis,Severe sepsis
3,3,13293,13293,13564,67020,Puerperal sepsis-unsp,"Puerperal sepsis, unspecified as to episode of..."
4,4,13294,13294,13565,67022,Puerprl sepsis-del w p/p,"Puerperal sepsis, delivered, with mention of p..."
5,5,13295,13295,13566,67024,Puerperl sepsis-postpart,"Puerperal sepsis, postpartum condition or comp..."


In [6]:
i = paths.index('DIAGNOSES_ICD.csv')
datas[i].sort_values(by=['SUBJECT_ID', 'SEQ_NUM'], inplace=True)
DIAGNOSES_ICD = datas[i].copy()
'''
drop_index = []
for j in tqdm(DIAGNOSES_ICD.index):
    for code in D_ICD_DIAGNOSES['ICD9_CODE']:
        if str(DIAGNOSES_ICD.loc[j]['ICD9_CODE']) == code:
            break
    else:
        drop_index.append(j)
DIAGNOSES_ICD.drop(drop_index, inplace=True)
del drop_index
DIAGNOSES_ICD.to_csv(prefix_path + paths[i])
'''
DIAGNOSES_ICD

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,747,140,21,111970,11.0,99592
1,914,307,38,185910,3.0,99592
2,1084,477,61,189535,8.0,99591
3,1097,490,62,116009,4.0,99591
4,276,505,64,172056,3.0,99591
...,...,...,...,...,...,...
5404,648640,650341,99836,101018,8.0,99592
5405,649024,650470,99865,189346,4.0,99592
5406,649529,650720,99912,189380,8.0,99592
5407,648171,650990,99985,176670,7.0,99592


In [7]:
'''
dict1 = defaultdict(list)
for i in range(len(dataset)):
    dict1[dataset['SUBJECT_ID'][i]].append(dataset['HADM_ID'][i])

for i in tqdm(range(len(datas))):
    if i == paths.index('DIAGNOSES_ICD.csv'):
        continue
    if 'SUBJECT_ID' in datas[i].columns and 'HADM_ID' in datas[i].columns:
        drop_index = []
        for j in datas[i].index:
            if datas[i]['HADM_ID'][j] not in dict1[datas[i]['SUBJECT_ID'][j]]:
                drop_index.append(j)
        datas[i].drop(drop_index, inplace=True)
        del drop_index
        # datas[i].sort_values(by=['SUBJECT_ID', 'HADM_ID'], inplace=True)
    elif 'SUBJECT_ID' in datas[i].columns:
        datas[i] = datas[i][[id in dataset['SUBJECT_ID'].values for id in datas[i]['SUBJECT_ID']]]
        # datas[i].sort_values(by=['SUBJECT_ID'], inplace=True)

    datas[i].to_csv(prefix_path + paths[i])
    
del dict1

        
i = paths.index('D_ICD_PROCEDURES.csv')
datas[i].sort_values(by=['ICD9_CODE'], inplace=True)
datas[i].to_csv(prefix_path + paths[i])

i = paths.index('D_ITEMS.csv')
datas[i].sort_values(by=['ITEMID'], inplace=True)
datas[i].to_csv(prefix_path + paths[i])

i = paths.index('D_LABITEMS.csv')
datas[i].sort_values(by=['ITEMID'], inplace=True)
datas[i].to_csv(prefix_path + paths[i])
'''
for i in tqdm(range(len(datas))):
    for column in datas[i].columns:
        if 'TIME' in column or 'DATE' in column or 'DOB' == column or 'DOD' == column or 'DOD_HOSP' == column or 'DOD_SSN' == column:
            datas[i][column] = datas[i][column].apply(lambda x : pd.Timestamp(x))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [03:26<00:00,  7.94s/it]


# Look Data characteristics

In [8]:
i = paths.index('ICUSTAYS.csv')
ICUSTAYS = datas[i].copy()
ICUSTAYS.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
1,7,372,275,129886,219649,carevue,CCU,CCU,7,7,2170-10-07 11:28:53,2170-10-14 14:38:07,7.1314
2,45,410,305,194340,217232,metavision,SICU,SICU,33,33,2129-09-03 12:31:31,2129-09-05 23:00:50,2.4370
3,67,432,323,143334,264375,carevue,MICU,MICU,23,23,2120-01-11 15:48:28,2120-01-14 16:24:43,3.0252
4,74,439,329,172132,217001,carevue,MICU,MICU,50,50,2154-04-05 16:36:51,2154-04-08 18:14:48,3.0680


In [9]:
i = paths.index('PATIENTS.csv')
PATIENTS = datas[i].copy()
PATIENTS.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,1,235,250,F,2164-12-27,2188-11-22,2188-11-22,NaT,1
1,18,252,269,M,2130-09-30,NaT,NaT,NaT,0
2,40,646,685,F,2126-07-29,NaT,NaT,NaT,0
3,44,650,689,F,2128-12-27,NaT,NaT,NaT,0
4,45,651,690,M,2109-09-24,2189-07-30,2189-07-30,2189-07-30,1


In [10]:
i = paths.index('ADMISSIONS.csv')
ADMISSIONS = datas[i].copy()
ADMISSIONS.drop(ADMISSIONS[ADMISSIONS['HAS_CHARTEVENTS_DATA'] == 0].index, inplace=True)
ADMISSIONS.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,19,40,38,185910,2166-08-10 00:28:00,2166-09-04 11:30:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,Medicare,NaN,CATHOLIC,WIDOWED,WHITE,NaT,NaT,ACUTE MYOCARDIAL INFARCTION-SEPSIS,0,1
1,23,457,357,145674,2198-08-02 04:49:00,2198-10-26 14:55:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2198-08-02 02:03:00,2198-08-02 05:37:00,SEIZURE,0,1
2,24,458,357,122609,2198-11-01 22:36:00,2198-11-14 14:20:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,2198-11-01 18:01:00,2198-11-01 23:06:00,SEPSIS,0,1
3,37,471,366,134462,2164-11-18 20:27:00,2164-11-22 15:18:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,SINGLE,HISPANIC OR LATINO,2164-11-18 10:52:00,2164-11-18 21:31:00,SEPSIS,0,1
4,63,61,61,189535,2119-01-04 18:12:00,2119-02-03 01:35:00,2119-02-03 01:35:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Private,NaN,CATHOLIC,MARRIED,WHITE,NaT,NaT,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,1,1


In [11]:
i = paths.index('INPUTEVENTS_CV.csv')
INPUTEVENTS_CV = datas[i].copy()
INPUTEVENTS_CV.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,ORDERID,LINKORDERID,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE
0,5257,7534,13168,182334.0,266561.0,2185-10-20 10:00:00,30056,120.0,ml,NaN,...,9164547,9164547,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
1,5258,7535,13168,182334.0,266561.0,2185-10-20 12:00:00,30056,150.0,ml,NaN,...,3692841,9164547,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
2,5259,7536,13168,182334.0,266561.0,2185-10-20 16:00:00,30056,100.0,ml,NaN,...,10641024,9164547,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
3,5260,7537,13168,182334.0,266561.0,2185-10-20 21:00:00,30056,100.0,ml,NaN,...,11331457,9164547,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
4,5261,7538,13168,182334.0,266561.0,2185-10-20 23:00:00,30056,60.0,ml,NaN,...,10791487,9164547,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN


In [12]:
i = paths.index('INPUTEVENTS_CV.csv')
INPUTEVENTS_MV = datas[i].copy()
INPUTEVENTS_MV.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,ORDERID,LINKORDERID,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE
0,5257,7534,13168,182334.0,266561.0,2185-10-20 10:00:00,30056,120.0,ml,NaN,...,9164547,9164547,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
1,5258,7535,13168,182334.0,266561.0,2185-10-20 12:00:00,30056,150.0,ml,NaN,...,3692841,9164547,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
2,5259,7536,13168,182334.0,266561.0,2185-10-20 16:00:00,30056,100.0,ml,NaN,...,10641024,9164547,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
3,5260,7537,13168,182334.0,266561.0,2185-10-20 21:00:00,30056,100.0,ml,NaN,...,11331457,9164547,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
4,5261,7538,13168,182334.0,266561.0,2185-10-20 23:00:00,30056,60.0,ml,NaN,...,10791487,9164547,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN


In [13]:
i = paths.index('OUTPUTEVENTS.csv')
OUTPUTEVENTS = datas[i].copy()
OUTPUTEVENTS.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR
0,63427,70086,13168,182334.0,266561.0,2185-10-23 06:30:00,40055,40.0,ml,2185-10-23 06:45:00,14442,NaN,NaN,NaN
1,63428,70087,13168,182334.0,266561.0,2185-10-23 08:00:00,40055,60.0,ml,2185-10-23 08:12:00,21570,NaN,NaN,NaN
2,63429,70088,13168,182334.0,266561.0,2185-10-23 09:00:00,40055,18.0,ml,2185-10-23 09:38:00,21570,NaN,NaN,NaN
3,63430,70089,13168,182334.0,266561.0,2185-10-23 11:00:00,40055,150.0,ml,2185-10-23 11:12:00,21570,NaN,NaN,NaN
4,63431,70090,13168,182334.0,266561.0,2185-10-23 15:00:00,40055,180.0,ml,2185-10-23 15:07:00,21570,NaN,NaN,NaN


In [14]:
i = paths.index('D_ITEMS.csv')
D_ITEMS = datas[i].copy()
D_ITEMS.head()

,Unnamed: 0,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,290,27,1,% Inspir. Time,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,291,28,2,ABI (L),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,292,29,3,ABI (R),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,293,30,4,ABI Ankle BP [Right],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,294,31,5,ABI Brachial BP [Right],NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [15]:
i = paths.index('D_LABITEMS.csv')
D_LABITEMS = datas[i].copy()
D_LABITEMS.head()

,Unnamed: 0,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
0,127,1,50800,SPECIMEN TYPE,BLOOD,BLOOD GAS,NaN
1,128,2,50801,Alveolar-arterial Gradient,Blood,Blood Gas,19991-9
2,129,3,50802,Base Excess,Blood,Blood Gas,11555-0
3,130,4,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas,1959-6
4,131,5,50804,Calculated Total CO2,Blood,Blood Gas,34728-6


In [16]:
i = paths.index('DRGCODES.csv')
DRGCODES = datas[i].copy()
DRGCODES.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
0,2895,2861,9102,154215,HCFA,385,"NEONATES, DIED OR TRANSFERRED TO ANOTHER ACUTE...",NaN,NaN
1,2967,2933,20617,178293,HCFA,386,EXTREME IMMATURITY OR RESPIRATORY DISTRESS SYN...,NaN,NaN
2,3053,3019,14831,160882,HCFA,386,EXTREME IMMATURITY OR RESPIRATORY DISTRESS SYN...,NaN,NaN
3,3764,3027,15204,168396,HCFA,386,EXTREME IMMATURITY OR RESPIRATORY DISTRESS SYN...,NaN,NaN
4,4396,4357,5424,157250,HCFA,386,EXTREME IMMATURITY OR RESPIRATORY DISTRESS SYN...,NaN,NaN


In [17]:
i = paths.index('LABEVENTS.csv')
LABEVENTS = datas[i].copy()
LABEVENTS.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,10085,10440,21,111970.0,51279,2135-02-03 05:04:00,2.81,2.81,m/uL,abnormal
1,10086,10441,21,111970.0,51301,2135-02-03 05:04:00,26.8,26.80,K/uL,abnormal
2,10087,10442,21,111970.0,50800,2135-02-03 05:13:00,MIX,NaN,NaN,NaN
3,10088,10443,21,111970.0,50813,2135-02-03 05:13:00,2.6,2.60,mmol/L,abnormal
4,10089,10444,21,111970.0,50817,2135-02-03 05:13:00,56,56.00,%,NaN


In [18]:
i = paths.index('PRESCRIPTIONS.csv')
PRESCRIPTIONS = datas[i].copy()
PRESCRIPTIONS.head()

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,435,1390475,21,111970,216859.0,2135-01-31,2135-01-31,BASE,NS (Mini Bag Plus),NaN,NaN,NS/MBP100I,001210,338055318.0,100mL Bag,100,ml,1,BAG,IV
1,436,1389751,21,111970,216859.0,2135-01-31,2135-01-31,MAIN,Levothyroxine Sodium,Levothyroxine Sodium,Levothyroxine Sodium,LEVO50,006649,74455211.0,50mcg Tablet,50,mcg,1,TAB,PO
2,437,1379358,21,111970,216859.0,2135-01-31,2135-01-31,MAIN,Meropenem,NaN,NaN,MERO500I,026488,310032520.0,500mg Vial,500,mg,1,VIAL,IV
3,438,1390474,21,111970,216859.0,2135-01-31,2135-02-01,BASE,SW,NaN,NaN,KCLBASE2,NaN,0.0,50ml Bag,100,ml,100,ml,IV
4,439,1390472,21,111970,216859.0,2135-01-31,2135-02-01,BASE,Iso-Osmotic Dextrose,NaN,NaN,VANCOBASE,NaN,0.0,200ml Bag,200,ml,200,ml,IV


In [19]:
i = paths.index('CHARTEVENTS1.csv')
CHARTEVENTS1 = datas[i].copy()
CHARTEVENTS2 = datas[i + 1].copy()
CHARTEVENTS3 = datas[i + 2].copy()
CHARTEVENTS4 = datas[i + 3].copy()
CHARTEVENTS5 = datas[i + 4].copy()
CHARTEVENTS6 = datas[i + 5].copy()
CHARTEVENTS7 = datas[i + 6].copy()
CHARTEVENTS8 = datas[i + 7].copy()
CHARTEVENTS9 = datas[i + 8].copy()
CHARTEVENTS1.head()

,Unnamed: 0.1,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
0,1635,1635,2190,85,112077,291697.0,223751,2167-07-26 16:58:00,2167-07-26 18:59:00,20214.0,160.0,160.0,mmHg,0.0,0.0,NaN,NaN
1,1636,1636,2191,85,112077,291697.0,223752,2167-07-26 16:58:00,2167-07-26 18:59:00,20214.0,90.0,90.0,mmHg,0.0,0.0,NaN,NaN
2,1637,1637,2192,85,112077,291697.0,223769,2167-07-26 16:58:00,2167-07-26 18:59:00,20214.0,100.0,100.0,%,0.0,0.0,NaN,NaN
3,1638,1638,2193,85,112077,291697.0,223770,2167-07-26 16:58:00,2167-07-26 18:59:00,20214.0,92.0,92.0,%,0.0,0.0,NaN,NaN
4,1639,1639,2194,85,112077,291697.0,224161,2167-07-26 16:58:00,2167-07-26 18:59:00,20214.0,35.0,35.0,insp/min,0.0,0.0,NaN,NaN


# Features

In [20]:
# last row don't understand meaning
features = ['Shock_Index', 'elixhauser', 'SIRS', 'gender', 're_admission', 'GCS', 'SOFA', 'age', 'Albumin',
           'Arterial_pH', 'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'SGPT', 
           'Arterial_blood_gas', 'BUN', 'HCO3', 'INR', 'Arterial_lactate', 'CO2', 'Creatinine', 
           'Ionised_Ca', 'PT', 'Platelets_count', 'SGOT', 'Total_bili', 'WBC_count', 'DiaBP', 'SysBP', 
           'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 'Weight_kg', 'HR', 'SpO2'
           'input_total_tev', 'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc', 'mechvent']

binary_features = ['gender', 're_admission', 'mechvent']
norm_features = ['Shock_Index', 'elixhauser', 'SIRS', 'GCS', 'SOFA', 'age', 'Albumin', 'Arterial_pH', 
                'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'Arterial_blood_gas', 
                'HCO3', 'Arterial_lactate', 'CO2', 'Ionised_Ca', 'PT', 'Platelets_count', 'WBC_count', 
                'DiaBP', 'SysBP', 'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 
                'Weight_kg', 'HR', ]
log_features = ['SGPT', 'BUN', 'INR', 'Creatinine', 'SGOT', 'Total_bili', 'SpO2', 'input_total_tev',
                'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc']

In [21]:
dataset = DIAGNOSES_ICD[['SUBJECT_ID', 'HADM_ID']].copy()
dataset.sort_values(by=['SUBJECT_ID', 'HADM_ID'], inplace=True)
dataset.index = range(len(dataset))
dataset

,SUBJECT_ID,HADM_ID
0,21,111970
1,38,185910
2,61,189535
3,62,116009
4,64,172056
...,...,...
5404,99836,101018
5405,99865,189346
5406,99912,189380
5407,99985,176670


## Add gender and age to dataset

In [22]:
dataset['DOB'] = np.nan
dataset['Gender'] = np.nan

for i in PATIENTS.index:
    subject_id = PATIENTS['SUBJECT_ID'].loc[i]
    gender = PATIENTS['GENDER'].loc[i]
    dob = PATIENTS['DOB'].loc[i]
    index = dataset[dataset['SUBJECT_ID'] == subject_id].index
    dataset['DOB'].loc[index] = dob
    dataset['Gender'].loc[index] = gender
    
dataset['Gender'] = dataset['Gender'].apply(lambda x : 1 if x == 'M' else 0)
dataset

,SUBJECT_ID,HADM_ID,DOB,Gender
0,21,111970,2047-04-04 00:00:00,1
1,38,185910,2090-08-31 00:00:00,1
2,61,189535,2063-10-21 00:00:00,1
3,62,116009,2044-05-08 00:00:00,1
4,64,172056,2116-06-27 00:00:00,0
...,...,...,...,...
5404,99836,101018,2064-04-13 00:00:00,0
5405,99865,189346,2107-06-25 00:00:00,0
5406,99912,189380,2068-09-26 00:00:00,1
5407,99985,176670,2127-04-08 00:00:00,1


## Add death time, admit time, discharge time to dataset

In [23]:
dataset['DEATHTIME'] = np.nan
dataset['ADMITTIME'] = np.nan
dataset['DISCHTIME'] = np.nan

for i in ADMISSIONS.index:
    subject_id = ADMISSIONS['SUBJECT_ID'].loc[i]
    hadm_id = ADMISSIONS['HADM_ID'].loc[i]
    death_time = ADMISSIONS['DEATHTIME'].loc[i]
    admittime = ADMISSIONS['ADMITTIME'].loc[i]
    dischtime = ADMISSIONS['DISCHTIME'].loc[i]
    index = dataset.query(f'SUBJECT_ID == {subject_id} & HADM_ID == {hadm_id}').index
    dataset['DEATHTIME'].loc[index] = death_time
    dataset['ADMITTIME'].loc[index] = admittime
    dataset['DISCHTIME'].loc[index] = dischtime
    
dataset

,SUBJECT_ID,HADM_ID,DOB,Gender,DEATHTIME,ADMITTIME,DISCHTIME
0,21,111970,2047-04-04 00:00:00,1,2135-02-08 02:08:00,2135-01-30 20:50:00,2135-02-08 02:08:00
1,38,185910,2090-08-31 00:00:00,1,NaT,2166-08-10 00:28:00,2166-09-04 11:30:00
2,61,189535,2063-10-21 00:00:00,1,2119-02-03 01:35:00,2119-01-04 18:12:00,2119-02-03 01:35:00
3,62,116009,2044-05-08 00:00:00,1,NaT,2113-02-15 00:19:00,2113-02-19 15:30:00
4,64,172056,2116-06-27 00:00:00,0,NaT,2143-03-03 09:25:00,2143-03-18 13:01:00
...,...,...,...,...,...,...,...
5404,99836,101018,2064-04-13 00:00:00,0,NaT,2116-05-21 19:23:00,2116-07-17 16:30:00
5405,99865,189346,2107-06-25 00:00:00,0,2184-12-30 00:01:00,2184-12-10 10:30:00,2184-12-30 00:01:00
5406,99912,189380,2068-09-26 00:00:00,1,NaT,2153-04-13 16:05:00,2153-04-27 13:40:00
5407,99985,176670,2127-04-08 00:00:00,1,NaT,2181-01-27 02:47:00,2181-02-12 17:05:00


## change time by ICUSTAY_ID in dataset

In [24]:
for i in ICUSTAYS.index:
    subject_id = ICUSTAYS['SUBJECT_ID'].loc[i]
    hadm_id = ICUSTAYS['HADM_ID'].loc[i]
    intime = ICUSTAYS['INTIME'].loc[i]
    outtime = ICUSTAYS['OUTTIME'].loc[i]
    index = dataset.query(f'SUBJECT_ID == {subject_id} & HADM_ID == {hadm_id}').index
    if dataset['ADMITTIME'].isnull()[index[0]]:
        dataset['ADMITTIME'].loc[index[0]] = intime
    elif dataset['ADMITTIME'].loc[index[0]] - intime > pd.Timedelta('00:00:00'):
        dataset['ADMITTIME'].loc[index[0]] = intime
    if dataset['DISCHTIME'].isnull()[index[0]]:
        dataset['DISCHTIME'].loc[index[0]] = outtime
    elif dataset['DISCHTIME'].loc[index[0]] - outtime < pd.Timedelta('00:00:00'):
        dataset['DISCHTIME'].loc[index[0]] = outtime


dataset.sort_values(by=['SUBJECT_ID', 'ADMITTIME'])


,SUBJECT_ID,HADM_ID,DOB,Gender,DEATHTIME,ADMITTIME,DISCHTIME
0,21,111970,2047-04-04 00:00:00,1,2135-02-08 02:08:00,2135-01-30 20:50:00,2135-02-08 05:38:46
1,38,185910,2090-08-31 00:00:00,1,NaT,2166-08-10 00:28:00,2166-09-04 13:39:23
2,61,189535,2063-10-21 00:00:00,1,2119-02-03 01:35:00,2119-01-04 18:12:00,2119-02-03 01:35:00
3,62,116009,2044-05-08 00:00:00,1,NaT,2113-02-15 00:19:00,2113-02-19 15:30:00
4,64,172056,2116-06-27 00:00:00,0,NaT,2143-03-03 09:25:00,2143-03-18 13:01:00
...,...,...,...,...,...,...,...
5404,99836,101018,2064-04-13 00:00:00,0,NaT,2116-05-21 19:23:00,2116-07-17 16:30:00
5405,99865,189346,2107-06-25 00:00:00,0,2184-12-30 00:01:00,2184-12-10 10:30:00,2184-12-31 02:56:46
5406,99912,189380,2068-09-26 00:00:00,1,NaT,2153-04-13 16:05:00,2153-04-27 13:40:00
5407,99985,176670,2127-04-08 00:00:00,1,NaT,2181-01-27 02:47:00,2181-02-12 17:05:00


## Add re-admission to dataset

In [25]:
dataset['re_admission'] = np.nan

dataset['re_admission'].loc[0] = 0
for i in dataset.index[1:]:
    if dataset['SUBJECT_ID'].loc[i] == dataset['SUBJECT_ID'].loc[i - 1]:
        dataset['re_admission'].loc[i] = 1
    else:
        dataset['re_admission'].loc[i] = 0

dataset

,SUBJECT_ID,HADM_ID,DOB,Gender,DEATHTIME,ADMITTIME,DISCHTIME,re_admission
0,21,111970,2047-04-04 00:00:00,1,2135-02-08 02:08:00,2135-01-30 20:50:00,2135-02-08 05:38:46,0.0
1,38,185910,2090-08-31 00:00:00,1,NaT,2166-08-10 00:28:00,2166-09-04 13:39:23,0.0
2,61,189535,2063-10-21 00:00:00,1,2119-02-03 01:35:00,2119-01-04 18:12:00,2119-02-03 01:35:00,0.0
3,62,116009,2044-05-08 00:00:00,1,NaT,2113-02-15 00:19:00,2113-02-19 15:30:00,0.0
4,64,172056,2116-06-27 00:00:00,0,NaT,2143-03-03 09:25:00,2143-03-18 13:01:00,0.0
...,...,...,...,...,...,...,...,...
5404,99836,101018,2064-04-13 00:00:00,0,NaT,2116-05-21 19:23:00,2116-07-17 16:30:00,0.0
5405,99865,189346,2107-06-25 00:00:00,0,2184-12-30 00:01:00,2184-12-10 10:30:00,2184-12-31 02:56:46,0.0
5406,99912,189380,2068-09-26 00:00:00,1,NaT,2153-04-13 16:05:00,2153-04-27 13:40:00,0.0
5407,99985,176670,2127-04-08 00:00:00,1,NaT,2181-01-27 02:47:00,2181-02-12 17:05:00,0.0


# Split time

In [26]:
dataset['Age'] = np.nan
for hour in range(1, 0, -1):
    hour_period = f'0{hour}:00:00'
    index = 0
    split = pd.DataFrame(columns = dataset.columns)
    for i in tqdm(dataset.index):
        intime, outtime = dataset['ADMITTIME'].loc[i], dataset['DISCHTIME'].loc[i]
        if dataset['ADMITTIME'].isnull()[i] or dataset['ADMITTIME'].isnull()[i]:
            continue
        while outtime - intime > pd.Timedelta(hour_period):
            #['SUBJECT_ID', 'HADM_ID', 'DOB', 'Gender', 'DEATHTIME', 'ADMITTIME', 'DISCHTIME', 're_admission', 'Age']
            split.loc[index] = [
                dataset['SUBJECT_ID'].loc[i],
                dataset['HADM_ID'].loc[i],
                dataset['DOB'].loc[i],
                dataset['Gender'].loc[i],
                dataset['DEATHTIME'].loc[i],
                intime,
                intime + pd.Timedelta(hour_period),
                dataset['re_admission'].loc[i],
                (intime.to_pydatetime() - dataset['DOB'].loc[i].to_pydatetime()).days / 365
            ]
            intime += pd.Timedelta(hour_period)
            index += 1

        split.loc[index] = [
            dataset['SUBJECT_ID'].loc[i],
            dataset['HADM_ID'].loc[i],
            dataset['DOB'].loc[i],
            dataset['Gender'].loc[i],
            dataset['DEATHTIME'].loc[i],
            intime,
            outtime,
            dataset['re_admission'].loc[i],
            (intime.to_pydatetime() - dataset['DOB'].loc[i].to_pydatetime()).days / 365
        ]

    split.drop('DOB', axis=1, inplace=True)
    split = split[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'Gender', 'Age', 'DEATHTIME', 're_admission']]
    split.columns = ['SUBJECT_ID', 'HADM_ID', 'STARTTIME', 'ENDTIME', 'Gender', 'Age', 'DEATHTIME', 're_admission']
    split.to_csv(f'../data/mimiciii/dataset_split_{hour}_hour.csv')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5409/5409 [11:35:03<00:00,  7.71s/it]


# Features

In [3]:
# last row don't understand meaning
features = ['Shock_Index', 'elixhauser', 'SIRS', 'gender', 're_admission', 'GCS', 'SOFA', 'age', 'Albumin',
           'Arterial_pH', 'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'SGPT', 
           'Arterial_blood_gas', 'BUN', 'HCO3', 'INR', 'Arterial_lactate', 'CO2', 'Creatinine', 
           'Ionised_Ca', 'PT', 'Platelets_count', 'SGOT', 'Total_bili', 'WBC_count', 'DiaBP', 'SysBP', 
           'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 'Weight_kg', 'HR', 'SpO2'
           'input_total_tev', 'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc', 'mechvent']

binary_features = ['gender', 're_admission', 'mechvent']
norm_features = ['Shock_Index', 'elixhauser', 'SIRS', 'GCS', 'SOFA', 'age', 'Albumin', 'Arterial_pH', 
                'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'Arterial_blood_gas', 
                'HCO3', 'Arterial_lactate', 'CO2', 'Ionised_Ca', 'PT', 'Platelets_count', 'WBC_count', 
                'DiaBP', 'SysBP', 'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 
                'Weight_kg', 'HR', ]
log_features = ['SGPT', 'BUN', 'INR', 'Creatinine', 'SGOT', 'Total_bili', 'SpO2', 'input_total_tev',
                'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc']

## Load require data

In [4]:
CHARTEVENTS1 = pd.read_csv(prefix_path + 'CHARTEVENTS1.csv')
CHARTEVENTS2 = pd.read_csv(prefix_path + 'CHARTEVENTS2.csv')
CHARTEVENTS3 = pd.read_csv(prefix_path + 'CHARTEVENTS3.csv')
CHARTEVENTS4 = pd.read_csv(prefix_path + 'CHARTEVENTS4.csv')
CHARTEVENTS5 = pd.read_csv(prefix_path + 'CHARTEVENTS5.csv')
CHARTEVENTS6 = pd.read_csv(prefix_path + 'CHARTEVENTS6.csv')
CHARTEVENTS7 = pd.read_csv(prefix_path + 'CHARTEVENTS7.csv')
CHARTEVENTS8 = pd.read_csv(prefix_path + 'CHARTEVENTS8.csv')
CHARTEVENTS9 = pd.read_csv(prefix_path + 'CHARTEVENTS9.csv')
LABEVENTS = pd.read_csv(prefix_path + 'LABEVENTS.csv')
OUTPUTEVENTS = pd.read_csv(prefix_path + 'OUTPUTEVENTS.csv')

datas = [CHARTEVENTS1, CHARTEVENTS2, CHARTEVENTS3, CHARTEVENTS4, CHARTEVENTS5, CHARTEVENTS6, CHARTEVENTS7, 
         CHARTEVENTS8, CHARTEVENTS9, LABEVENTS, OUTPUTEVENTS]

for i in tqdm(range(len(datas))):
    for column in datas[i].columns:
        if 'TIME' in column or 'DATE' in column or 'DOB' == column or 'DOD' == column or 'DOD_HOSP' == column or 'DOD_SSN' == column:
            datas[i][column] = datas[i][column].apply(lambda x : pd.Timestamp(x))

del datas

/tmp/ipykernel_44179/2293973746.py:1: DtypeWarning: Columns (10,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  CHARTEVENTS1 = pd.read_csv(prefix_path + 'CHARTEVENTS1.csv')
/tmp/ipykernel_44179/2293973746.py:5: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  CHARTEVENTS5 = pd.read_csv(prefix_path + 'CHARTEVENTS5.csv')
/tmp/ipykernel_44179/2293973746.py:7: DtypeWarning: Columns (10,12,16) have mixed types. Specify dtype option on import or set low_memory=False.
  CHARTEVENTS7 = pd.read_csv(prefix_path + 'CHARTEVENTS7.csv')
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [03:02<00:00, 16.58s/it]


## Load variable range

In [5]:
var_range = pd.read_csv('../data/mimiciii/variable_range.csv')
var_range.index = var_range.iloc[:, 0]
var_range = var_range.iloc[:, 1:]
var_range['GROUP_ID'] = var_range['GROUP_ID'].apply(lambda x : [int(s) for s in x.split(',')] if type(x) == type('') else x)
var_range

,OUTLIER LOW,VALID LOW,IMPUTE,VALID HIGH,OUTLIER HIGH,GROUP_ID
Unnamed: 0,,,,,,
GCS,0.0,0.00,NaN,15.0,15.0,[198]
SOFA,0.0,0.00,NaN,24.0,24.0,[227428]
Albumin,0.0,0.60,3.10,6.0,60.0,"[50862, 772, 1521, 227456]"
Arterial pH,NaN,NaN,NaN,NaN,NaN,"[780, 1126, 223830, 50820]"
Calcium,NaN,NaN,NaN,NaN,NaN,"[225625, 50893, 786, 1522]"
Glucose,0.0,33.00,128.00,2000.0,2200.0,"[220621, 225664, 50809, 811, 807, 50931, 22653..."
Hemoglobin,0.0,0.00,10.20,25.0,30.0,"[220228, 51222, 50811, 814]"
Magnesium,0.0,0.00,2.00,20.0,22.0,"[220635, 50960, 1532, 821]"
PTT,0.0,18.80,34.30,150.0,150.0,"[227466, 825, 1533, 51275]"


## Merge features to dataset

In [33]:
def merge(dataset, feature, EVENTS, var_range, isOutputEvent = False):
    # find all feature records in EVENTS
    event = EVENTS[[id in var_range.loc[feature]['GROUP_ID'] for id in EVENTS['ITEMID']]]

    for i in range(len(event)):
        
        subject_id = event.iloc[i]['SUBJECT_ID']
        hadm_id = event.iloc[i]['HADM_ID']
        time = event.iloc[i]['CHARTTIME']
        
        # detect outlier value, if out of outlier range then set to missing value
        if isOutputEvent:
            value = event.iloc[i]['VALUE']
        else:
            value = event.iloc[i]['VALUENUM']
        if var_range.loc[feature].notnull()[0]:
            if value > var_range.loc[feature]['OUTLIER HIGH']:
                value = np.nan
            elif value > var_range.loc[feature]['VALID HIGH']:
                value = var_range.loc[feature]['VALID HIGH']
            elif value < var_range.loc[feature]['OUTLIER LOW']:
                value = np.nan
            elif value < var_range.loc[feature]['VALID LOW']:
                value = var_range.loc[feature]['VALID LOW']
        
        index = dataset.query(f'SUBJECT_ID == {subject_id} & HADM_ID == {hadm_id}').index
        
        for idx in index:
            if time - dataset.iloc[idx]['STARTTIME'] >= pd.Timedelta('0') and time - dataset.iloc[idx]['ENDTIME'] <= pd.Timedelta('0'):
                if dataset['records'][idx] == 0:
                    dataset[feature][idx] = value
                else:
                    record = dataset['records'][idx]
                    # use mean when multiple records in same period
                    dataset[feature][idx] = value / (record + 1) + record / (record + 1) * dataset[feature][idx]
                dataset['records'][idx] += 1
                break
                    
for hour in tqdm(range(4, 0, -1)):
    path = f'../data/mimiciii/dataset_split_{hour}_hour.csv'

    dataset = pd.read_csv(path)

    # change time column data's type to timestamp type
    dataset['STARTTIME'] = dataset['STARTTIME'].apply(lambda x : pd.Timestamp(x))
    dataset['ENDTIME'] = dataset['ENDTIME'].apply(lambda x : pd.Timestamp(x))
    dataset['DEATHTIME'] = dataset['DEATHTIME'].apply(lambda x : pd.Timestamp(x))
    dataset = dataset.loc[:, ['SUBJECT_ID', 'HADM_ID', 'STARTTIME', 'ENDTIME', 'Gender', 
                              'Age', 'DEATHTIME', 're_admission']]
    # dataset.head()                    


    for feature in tqdm(var_range.index):
        # count records in same period
        dataset['records'] = pd.Series([0] * len(dataset), dtype = np.int64)

        # add new feature column
        dataset[feature] = pd.Series([np.nan] * len(dataset), dtype = np.float64)

        # add data to new feature column with corresponding subject_id, hadm_id, icustay_id and time
        merge(dataset, feature, LABEVENTS, var_range)
        merge(dataset, feature, OUTPUTEVENTS, var_range, isOutputEvent = True)
        merge(dataset, feature, CHARTEVENTS1, var_range)
        merge(dataset, feature, CHARTEVENTS2, var_range)
        merge(dataset, feature, CHARTEVENTS3, var_range)
        merge(dataset, feature, CHARTEVENTS4, var_range)
        merge(dataset, feature, CHARTEVENTS5, var_range)
        merge(dataset, feature, CHARTEVENTS6, var_range)
        merge(dataset, feature, CHARTEVENTS7, var_range)
        merge(dataset, feature, CHARTEVENTS8, var_range)
        merge(dataset, feature, CHARTEVENTS9, var_range)

    dataset.to_csv(f'../data/mimiciii/dataset_split_{hour}_hour_after_merge.csv')

  3%|████▋                                                                                                                                                           | 1/34 [1:17:26<42:35:46, 4646.87s/it]


 25%|███████████████████████████████████████▎                                                                                                                     | 1/4 [76:24:52<229:14:38, 275092.89s/it]


KeyboardInterrupt: 

### shock index = HR / SBP (Heart rate / systolic blood pressure)

### elixhauser

### SIRS (Use diagnosis? or use criteria ?)

### PaO2/FiO2 ratio

### Mechanical Ventilation (in produreevents_mv)